Click the button above to load the interactive widgets

In [1]:
import pandas as pd
import numpy as np
from scipy.integrate import odeint
import matplotlib.pyplot as plt
from datetime import datetime,timedelta
from sklearn.metrics import mean_squared_error
from scipy.optimize import curve_fit, fsolve

from IPython.display import Image
from ipywidgets import interact, fixed, ToggleButtons, IntSlider, FloatSlider, Dropdown, SelectMultiple, Layout

from pandas.tseries.offsets import Week
%matplotlib inline

<b> Preface </b>
    
    Here are being condensed few simple models and data visualization tools that capture my attention in literature. The idea is to play around with it and understand how important is to limit social contact in this period, and also the importance of modelling for understanding and timely actions.

# Load data from JHU

In [2]:
def loadData(fileName):
    data = pd.read_csv(baseURL + fileName) \
             .drop(['Lat', 'Long'], axis=1) #\
#              .melt(id_vars=['Province/State', 'Country/Region'], 
#                  var_name='date', value_name=columnName) \
#              .astype({'date':'datetime64[ns]', columnName:'Int64'}, 
#                  errors='ignore')
    data['Province/State'].fillna('<all>', inplace=True)
#     data[columnName].fillna(0, inplace=True)
    return data

In [3]:
baseURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/"

In [4]:
Confirmed = loadData("time_series_covid19_confirmed_global.csv")

# Structure dataset

In [5]:
# grouping by country
Confirmed = Confirmed.groupby(['Country/Region']).sum()

In [6]:
Confirmed = Confirmed.T

In [7]:
Confirmed.index = pd.to_datetime(Confirmed.index)

# SIR Model definition

Inspired by https://www.linkedin.com/pulse/covid-19-from-model-prediction-predictive-control-ingmar-nopens/

In [8]:
def model_der(variables, t, beta, gamma):

    S = variables[0]
    I = variables[1]
    R = variables[2]
    
    S_new = -beta*I*S 
    I_new = beta*I*S - gamma*I
    R_new = gamma*I
    return [S_new, I_new, R_new]

In [9]:
def SIRmodel_logbeta2(timesteps, S_0, I_0, R_0, log10_beta, gamma, measured_cases, country, returnDataFrame=True, plotFig=True):
    """
    implementation of SIR population model
    
    Parameters
    -----------
    timesteps : np.array
    country: string
    log10_beta: float <0
    gamma: float
    
    """
    
    modeloutput = odeint(model_der, [S_0, I_0, R_0], timesteps, args=(10**(log10_beta), gamma));
    modeloutput = pd.DataFrame(modeloutput, columns=['S','I','R'], index=timesteps)
    
    modeloutput.index = pd.date_range(start='1/20/2020', periods=len(timesteps))
    
    if plotFig:
        fig, axes = plt.subplots(1,1, figsize=(10,8))
        axes.set_xlabel('time')
        axes.set_ylabel('Population')

        axes.plot(measured_cases[country], 'rx')
        axes.plot(modeloutput);
#         measured_cases[country].plot(axes=axes)
        
    if returnDataFrame:
        return modeloutput 

# Run SIR model

Try to fir the SIR model to the cases of a specific country and compare it to the logistic model

In [11]:
interact(SIRmodel_logbeta2, 
         
         timesteps=fixed(np.arange(0., 1000, 1)), 
         
         country = Dropdown(options=list(Confirmed.columns), value='US'),
         
         measured_cases = fixed(Confirmed),
         
         S_0=FloatSlider(value=500000, 
                           min=Confirmed.max().mean(), 
                           max=5*Confirmed.max().max(), 
                           step=1000,
                           continuous_update=False,
                           description='Affected population'), 
         I_0=fixed(1), R_0=fixed(0),
         
         log10_beta=FloatSlider(value=-6.3, min=-8, max=-5, step=0.001,
                                continuous_update=True, readout_format='.2e',
                               description='log10(infection rate)'),
         
         gamma=FloatSlider(value=0.06, min=0, max=0.06, step=0.001,
                           continuous_update=False, readout_format='.2e',
                          description='Recovery rate'),
         returnDataFrame=fixed(False));
# axes.plot(COVID19_totalConf, 'rx')

interactive(children=(FloatSlider(value=500000.0, continuous_update=False, description='Affected population', …

# LogLog visualization (when are we on our way out of pandemic)

Inspired by https://t.co/q2AtUQBX5P?amp=1

In [14]:
def infectionrate(Confirmed, country, return_plot = True, return_df=False):
    """
    Confirmed: df adjusted from JHU
    country: string
    """
    country = list(country)
    res = (Confirmed[country] - Confirmed[country].shift(1, freq=Week()).reindex(Confirmed[country].index)).fillna(value=0)
    if return_plot is True:
        res.plot()
    if return_df is True:
        return(res);

In [34]:
def loglog_covid(Confirmed, countries):
    """
    Confirmed: df adjustedc from JHU
    countries: list in columns of Confirmed
    
    """
    fig, axes = plt.subplots(figsize=(15,8))
    
    df = infectionrate(Confirmed, countries, return_plot=False, return_df=True)

    axes.loglog(Confirmed.sum(axis=1), df.values, 'o')
    axes.legend(countries)
    axes.set_title('Loglog')
    axes.set_ylabel('Weekly increase of positive cases')
    axes.set_xlabel('Total cases globally')
    axes.set_xlim([1e4,1e7])

## Visualization of infection rate (weekly new cases)

Check the rate of infections for one or multiple countries

In [35]:
interact(infectionrate,
         Confirmed = fixed(Confirmed),
         country = SelectMultiple(options=list(Confirmed.columns), value=['Belgium', 'Italy', 'Spain', 'US'],
                                  description='Select country(ies)',
                                  style = {'description_width': 'initial'},
                                  layout=Layout(width='40%', height='100px')),
         return_df = fixed(False),
         return_plot = fixed(True));

interactive(children=(SelectMultiple(description='Select country(ies)', index=(16, 84, 153, 168), layout=Layou…

## loglog plot

Inspired by https://t.co/q2AtUQBX5P?amp=1

A smart way of visualizing how effective are the measures taken from each country to limit the infection rate. When the Y axis values drop significantly, this means that the spreading of the infections is sensibly decreased.

In [37]:
interact(loglog_covid,
         Confirmed = fixed(Confirmed),
         countries = SelectMultiple(options=list(Confirmed.columns), value=['Belgium', 'Italy', 'Spain', 'US'],
                                 description='Select countries (Ctrl+ for multi-selection)', 
                                    style = {'description_width': 'initial'},
                                    layout=Layout(width='50%', height='200px')),
         return_df= fixed(False));

interactive(children=(SelectMultiple(description='Select countries (Ctrl+ for multi-selection)', index=(16, 84…